En este notebook vamos a armar y entrenar los primeros modelos en base al set creado en EDA.ipynb

In [1]:
from src import metrics
from src import plots
from src import preprocessing
from src import models

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.base import clone

In [2]:
df = pd.read_csv("data/processed/monaco_2025_colapinto_alllaps.csv")

# Target
y = df["LapTime_s"].to_numpy()

# Features legales
LEGAL_FEATURES_NUM = ["LapNumber", "Stint", "TyreLife"]
LEGAL_FEATURES_CAT = ["Session", "Compound"]

LEGAL_FEATURES_NUM = [c for c in LEGAL_FEATURES_NUM if c in df.columns]
LEGAL_FEATURES_CAT = [c for c in LEGAL_FEATURES_CAT if c in df.columns]

X = df[LEGAL_FEATURES_NUM + LEGAL_FEATURES_CAT].copy()



In [3]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), LEGAL_FEATURES_NUM),
        ("cat", OneHotEncoder(handle_unknown="ignore"), LEGAL_FEATURES_CAT),
    ]
)


Analisis de NaNs en los datos

In [4]:
FEATURES = LEGAL_FEATURES_NUM + LEGAL_FEATURES_CAT

missing_counts = df[FEATURES].isna().sum()
missing_percent = df[FEATURES].isna().mean() * 100

na_summary = pd.DataFrame({
    "missing_count": missing_counts,
    "missing_%": missing_percent.round(2)
}).sort_values("missing_%", ascending=False)

na_summary


,missing_count,missing_%
LapNumber,0,0.0
Stint,0,0.0
TyreLife,0,0.0
Session,0,0.0
Compound,0,0.0


In [5]:
for col in FEATURES:
    na_mask = df[col].isna()
    if na_mask.any():
        print(f"\nColumna: {col}")
        print(df.loc[na_mask, "Session"].value_counts())


In [6]:
cols_context = ["Session", "LapNumber", "Stint", "Compound", "TyreLife"]

for col in FEATURES:
    na_mask = df[col].isna()
    if na_mask.any():
        print(f"\n=== Ejemplos de filas con NaN en {col} ===")
        display(df.loc[na_mask, cols_context].head(10))
    else:
        print(f"\nNo hay NaN en la columna {col}")



No hay NaN en la columna LapNumber

No hay NaN en la columna Stint

No hay NaN en la columna TyreLife

No hay NaN en la columna Session

No hay NaN en la columna Compound


Hacemos CrossValidation con K-Fold para poder tener una mejor evaluacion.

In [7]:
def evaluate_model_cv(name, regressor, X, y, n_splits=5, random_state=42):
    """
    Entrena y evalúa un modelo de regresión usando KFold CV.
    Devuelve un dict con métricas promedio.
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    mae_scores = []
    rmse_scores = []
    r2_scores = []
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(X), start=1):
        X_train = X.iloc[train_idx]
        X_test  = X.iloc[test_idx]
        y_train = y[train_idx]
        y_test  = y[test_idx]
        
        # Nuevo pipeline para este fold
        reg = clone(regressor)
        model = Pipeline(steps=[
            ("preprocess", preprocessor),
            ("regressor", reg),
        ])
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        mae = metrics.MAE(y_test, y_pred)
        rmse = metrics.RMSE(y_test, y_pred)
        r2 = metrics.R2(y_test, y_pred)
        
        mae_scores.append(mae)
        rmse_scores.append(rmse)
        r2_scores.append(r2)
        
        print(f"[{name}] Fold {fold}: MAE={mae:.3f}, RMSE={rmse:.3f}, R2={r2:.3f}")
    
    mae_mean, mae_std = np.mean(mae_scores), np.std(mae_scores)
    rmse_mean, rmse_std = np.mean(rmse_scores), np.std(rmse_scores)
    r2_mean, r2_std = np.mean(r2_scores), np.std(r2_scores)
    
    print(f"\n[{name}] === Promedio {n_splits} folds ===")
    print(f"MAE  medio: {mae_mean:.3f} ± {mae_std:.3f}")
    print(f"RMSE medio: {rmse_mean:.3f} ± {rmse_std:.3f}")
    print(f"R2   medio: {r2_mean:.3f} ± {r2_std:.3f}\n")
    
    return {
        "model": name,
        "MAE_mean": mae_mean,
        "MAE_std": mae_std,
        "RMSE_mean": rmse_mean,
        "RMSE_std": rmse_std,
        "R2_mean": r2_mean,
        "R2_std": r2_std,
    }

Defino 4 Primeros modelos para seleccionar uno como Baseline y poder realizar Feature Engineering.

Entreno un RandomForest, un GradientBoosting, un Riedge y un MLP

In [8]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor

models = {
    "RandomForest": RandomForestRegressor(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    ),
    "GradientBoosting": GradientBoostingRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=3,
        random_state=42
    ),
    "Ridge": Ridge(alpha=1.0, random_state=42),

    "MLP": MLPRegressor(
        hidden_layer_sizes=(64, 32),
        activation="relu",
        solver="adam",
        learning_rate_init=1e-3,
        max_iter=500,
        random_state=42
    ),
}


In [9]:
import pandas as pd

results = []

for name, reg in models.items():
    res = evaluate_model_cv(name, reg, X, y, n_splits=5, random_state=42)
    results.append(res)

results_df = pd.DataFrame(results)



[RandomForest] Fold 1: MAE=0.936, RMSE=1.346, R2=0.657
[RandomForest] Fold 2: MAE=0.386, RMSE=0.466, R2=0.965
[RandomForest] Fold 3: MAE=0.718, RMSE=1.009, R2=0.823
[RandomForest] Fold 4: MAE=0.954, RMSE=1.353, R2=0.529
[RandomForest] Fold 5: MAE=0.557, RMSE=0.709, R2=0.825

[RandomForest] === Promedio 5 folds ===
MAE  medio: 0.710 ± 0.219
RMSE medio: 0.977 ± 0.350
R2   medio: 0.760 ± 0.151

[GradientBoosting] Fold 1: MAE=0.955, RMSE=1.306, R2=0.677
[GradientBoosting] Fold 2: MAE=0.370, RMSE=0.461, R2=0.966
[GradientBoosting] Fold 3: MAE=0.713, RMSE=0.990, R2=0.829
[GradientBoosting] Fold 4: MAE=0.983, RMSE=1.366, R2=0.520
[GradientBoosting] Fold 5: MAE=0.501, RMSE=0.621, R2=0.866

[GradientBoosting] === Promedio 5 folds ===
MAE  medio: 0.704 ± 0.242
RMSE medio: 0.949 ± 0.360
R2   medio: 0.772 ± 0.156

[Ridge] Fold 1: MAE=1.074, RMSE=1.360, R2=0.650
[Ridge] Fold 2: MAE=0.752, RMSE=0.979, R2=0.847
[Ridge] Fold 3: MAE=1.048, RMSE=1.300, R2=0.706
[Ridge] Fold 4: MAE=1.259, RMSE=1.622, R2=

d:\Archivos\UdeSA\IngIA\2025\Machine Learning\TpFinal-F1\F1_Pini_Carruthers\pytorch_cpu_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[MLP] Fold 1: MAE=3.561, RMSE=4.241, R2=-2.405


d:\Archivos\UdeSA\IngIA\2025\Machine Learning\TpFinal-F1\F1_Pini_Carruthers\pytorch_cpu_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[MLP] Fold 2: MAE=2.457, RMSE=3.047, R2=-0.480


d:\Archivos\UdeSA\IngIA\2025\Machine Learning\TpFinal-F1\F1_Pini_Carruthers\pytorch_cpu_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[MLP] Fold 3: MAE=2.102, RMSE=2.919, R2=-0.482


d:\Archivos\UdeSA\IngIA\2025\Machine Learning\TpFinal-F1\F1_Pini_Carruthers\pytorch_cpu_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[MLP] Fold 4: MAE=3.689, RMSE=4.472, R2=-4.143
[MLP] Fold 5: MAE=2.690, RMSE=3.153, R2=-2.457

[MLP] === Promedio 5 folds ===
MAE  medio: 2.900 ± 0.622
RMSE medio: 3.566 ± 0.653
R2   medio: -1.993 ± 1.384



d:\Archivos\UdeSA\IngIA\2025\Machine Learning\TpFinal-F1\F1_Pini_Carruthers\pytorch_cpu_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Resumen de las metricas de los Modelos evaluados por Cross-Validation:

In [10]:
results_df

,model,MAE_mean,MAE_std,RMSE_mean,RMSE_std,R2_mean,R2_std
0,RandomForest,0.710093,0.218863,0.976712,0.349585,0.759938,0.151176
1,GradientBoosting,0.704468,0.242255,0.949001,0.360163,0.771717,0.156358
2,Ridge,0.974944,0.200098,1.270406,0.223421,0.622526,0.172634
3,MLP,2.899803,0.622210,3.566432,0.653400,-1.993208,1.384126


Seleccion de Modelo Baseline:

GradientBoosting es el mejor basicamente en todas las metricas o muy similares a las del RF. La diferencia no es enorme, pero si tengo que elegir uno, el GB gana.


- Tiende a generalizar un poco mejor,
- Suele ser más sensible a pequeños cambios de features (bueno para ver el efecto del feature engineering).
- Es buen candidato para tunear despues ya que puedo jugar con n_estimators, learning_rate, max_depth, etc.

Con respecto al resto de modelos
- RandomForest: Está muy cerca en performance. Lo usaría como segundo modelo de comparación.
- Ridge: Me puede servir para ver cuánto aportan las relaciones no lineales y el feature engineering. Si con nuevas features Ridge mejora mucho, se que estoy agregando información “linealmente útil”.
- MLP: Lo descartaría.Con pocos datos y sin tuning claramente está haciendo overfitting o underfitting y con errores gigantes.

Empeizo con el Feature Engineering:

Features a crear:
(Armar lista y explicar cada una)

Uso add_basic_features() de Preproccessing.py para agregar las features nuevas.

In [11]:
df = pd.read_csv("data/processed/monaco_2025_colapinto_alllaps.csv")

# Aplicar feature engineering v1
df_fe = preprocessing.add_basic_features(df)

# Target
y = df_fe["LapTime_s"].to_numpy()


Redefino las columnas a utilizar:

In [15]:
LEGAL_FEATURES_NUM = [
    "LapNumber",
    "Stint",
    "TyreLife",
    "lap_norm_session",
    "stint_len",
    "stint_lap_index",
    "stint_lap_norm",
    "tyrelife_norm_stint",
    "is_race",
    "compound_order",
]

LEGAL_FEATURES_CAT = [
    "Session",
    "Compound",
]

LEGAL_FEATURES_NUM = [c for c in LEGAL_FEATURES_NUM if c in df_fe.columns]
LEGAL_FEATURES_CAT = [c for c in LEGAL_FEATURES_CAT if c in df_fe.columns]

FEATURES = LEGAL_FEATURES_NUM + LEGAL_FEATURES_CAT
X = df_fe[FEATURES].copy()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), LEGAL_FEATURES_NUM),
        ("cat", OneHotEncoder(handle_unknown="ignore"), LEGAL_FEATURES_CAT),
    ]
)


#printeo las columnas de x
print(X.columns)

Index(['LapNumber', 'Stint', 'TyreLife', 'lap_norm_session', 'stint_len',
       'stint_lap_index', 'stint_lap_norm', 'tyrelife_norm_stint', 'is_race',
       'compound_order', 'Session', 'Compound'],
      dtype='object')


In [13]:
# Definimos el modelo base de Gradient Boosting
gb_fe = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)

# Evaluamos con KFold usando las nuevas features
gb_fe_results = evaluate_model_cv(
    name="GradientBoosting_FE_v1",
    regressor=gb_fe,
    X=X,
    y=y,
    n_splits=5,
    random_state=42
)




[GradientBoosting_FE_v1] Fold 1: MAE=0.850, RMSE=1.265, R2=0.697
[GradientBoosting_FE_v1] Fold 2: MAE=0.321, RMSE=0.397, R2=0.975
[GradientBoosting_FE_v1] Fold 3: MAE=0.864, RMSE=1.103, R2=0.789
[GradientBoosting_FE_v1] Fold 4: MAE=1.157, RMSE=1.698, R2=0.258
[GradientBoosting_FE_v1] Fold 5: MAE=0.532, RMSE=0.692, R2=0.834

[GradientBoosting_FE_v1] === Promedio 5 folds ===
MAE  medio: 0.745 ± 0.290
RMSE medio: 1.031 ± 0.452
R2   medio: 0.711 ± 0.243



Resumen de los Resultados del FE_v1:

Todas las metricas empeoraron. Por qué puede haber pasado: 
- Dataset chico + más features = más riesgo de sobreajuste / ruido
- Metimos features muy derivadas de las mismas cosas
- Hay features que, desde la lógica del simulador, usan “info del futuro” (Bastante trampa)

Vamos a hacer una segunda version Feature Engineering v2 bastante mas conservadora.

Feature Engineering v2:

Elimino estas features “con futuro” del DataFrame y de LEGAL_FEATURES_NUM:
- stint_len
- stint_lap_index
- stint_lap_norm
- tyrelife_norm_stint

Me quedo con las que conceptualmente sí tienen sentido para el simulador y no duplican demasiado:

- lap_norm_session → fase de la sesión (principio/medio/fin).
- is_race → modo práctica vs carrera (puede ser útil).
- compound_order → codifica “blando vs duro” de forma ordenada.

In [16]:
LEGAL_FEATURES_NUM = [
    "LapNumber",
    "Stint",
    "TyreLife",
    "lap_norm_session",
    "is_race",
    "compound_order",
]

LEGAL_FEATURES_CAT = [
    "Session",
    "Compound",
]


In [ ]:
LEGAL_FEATURES_NUM = [c for c in LEGAL_FEATURES_NUM if c in df_fe.columns]
LEGAL_FEATURES_CAT = [c for c in LEGAL_FEATURES_CAT if c in df_fe.columns]

FEATURES = LEGAL_FEATURES_NUM + LEGAL_FEATURES_CAT
X = df_fe[FEATURES].copy()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), LEGAL_FEATURES_NUM),
        ("cat", OneHotEncoder(handle_unknown="ignore"), LEGAL_FEATURES_CAT),
    ]
)

#printeo las columnas de x
print(X.columns)

Index(['LapNumber', 'Stint', 'TyreLife', 'lap_norm_session', 'is_race',
       'compound_order', 'Session', 'Compound'],
      dtype='object')


In [ ]:
# Evaluamos con KFold usando las nuevas features
gb_fe_results = evaluate_model_cv(
    name="GradientBoosting_FE_v2",
    regressor=gb_fe,
    X=X,
    y=y,
    n_splits=5,
    random_state=42
)

[GradientBoosting_FE_v2] Fold 1: MAE=0.803, RMSE=1.113, R2=0.766
[GradientBoosting_FE_v2] Fold 2: MAE=0.408, RMSE=0.522, R2=0.957
[GradientBoosting_FE_v2] Fold 3: MAE=0.642, RMSE=0.908, R2=0.856
[GradientBoosting_FE_v2] Fold 4: MAE=1.026, RMSE=1.527, R2=0.400
[GradientBoosting_FE_v2] Fold 5: MAE=0.548, RMSE=0.641, R2=0.857

[GradientBoosting_FE_v2] === Promedio 5 folds ===
MAE  medio: 0.685 ± 0.213
RMSE medio: 0.942 ± 0.358
R2   medio: 0.767 ± 0.193



Resumen de los Resultados para FE V2:

- En comparacion con el modelo de GB sin FE se mejora un poco el MAE y el RMSE, manteniendo valor practicamente igual de R2
- FE_v2 no rompe nada y ayuda un poco
- Las nuevas features son conceptualmente correctas para el simulador
- El modelo sigue explicando ~75% de la varianza de LapTime (baseline sólido para comparar estrategias “gruesas” en el simulador y, más adelante, ver si el aprendizaje de (PCA/AE) aporta algo extra.)



Pasos a seguir:
- Congelar este setup como baseline oficial: Features: LapNumber, Stint, TyreLife, lap_norm_session, is_race, compound_order, Session, Compound. Modelo: GradientBoosting con los hiperparámetros actuales. Guardar estos resultados (tabla y configuración) para la parte del informe/paper (“Baseline clásico”).
-  Hacer tuning ligero de hiperparámetros del GB (n_estimators, max_depth, learning_rate) usando el mismo K-fold.

Mas adelante:
- armar el pipeline de PCA + clustering sobre las vueltas
- diseñar el autoencoder para aprender el espacio latente y ver si aparecen clusters por piloto/compuesto/estado de pista.